# Interactive plotting

When working within a notebook, you can use the interactive version of Matplotlib combined with the ```ipywidgets``` package to create interactive animations of the analysis. Here we reproduce the analyis shown in the [main documentation](Intensity_by_sector.ipynb) and then create such an interactive plot.

The first important step is to activate the interactive version of Matplotlib:

In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io

import matplotlib
cmap = matplotlib.colors.ListedColormap(np.random.rand(256,3))

from microfilm.dataset import MultipageTIFF
import microfilm.microplot
from splitmask import splitmask

import ipywidgets as ipw

Then we run the main analysis as usual:

In [6]:
image_path = '../splitmask/tests/test_folders/test_multipage_good/'
roi_path = '../splitmask//tests/test_folders/test_multipage_good/roi.bmp'

image = MultipageTIFF(image_path)
roi = skimage.io.imread(roi_path)

cm = splitmask.get_roi_cm(roi_im=roi)

im_np = image.load_frame(channel_name=image.channel_name[0], frame=20)

angular_width = 20
max_rad =50
sector_labels = splitmask.create_sector_mask(center=cm, im_dims=image.dims,
                                                     angular_width=angular_width, max_rad=max_rad)
colors_angle, cmap_angle = splitmask.get_cmap_labels(sector_labels, cmap_name='viridis')

channel = image.channel_name[1]

signal_angle = splitmask.measure_intensities(time_image=image.frame_generator(channel=channel), 
                                                     im_labels=sector_labels,max_time=300)


Finally, we use the ```interactive``` function of ```ipywidgets``` to update the plot when moving a slider:

In [8]:
fig, ax = plt.subplots(1,2, figsize=(8,4))
splitmask.plot_sectors(image, sector_labels, cmap=cmap_angle, ax=ax[0], channel=image.channel_name[1])
splitmask.plot_signals(signal_angle, color_array=colors_angle, ax=ax[1])  
time_line = ax[1].plot([0,0],[signal_angle.min(), signal_angle.max()],'-r')
plt.tight_layout()

def plot_sectors_time(t):
    ax[0].get_images()[0].set_data(image.load_frame(image.channel_name[1],t)) 
    time_line[0].set_xdata([t,t])
    
ipw.interactive(plot_sectors_time, t=ipw.IntSlider(value=0, min=0, max=image.max_time-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=50), Output()), _dom_classes=('widget-interact',…